In [6]:
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.les_mis import data
import numpy as np
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts

from random import random
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource, Range1d



hv.extension('bokeh')

nodes = data['nodes']
names = [node['name'] for node in sorted(data['nodes'], key=lambda x: x['group'])]

N = len(nodes)
counts = np.zeros((N, N))
for link in data['links']:
    counts[link['source'], link['target']] = link['value']
    counts[link['target'], link['source']] = link['value']

colormap = ["#444444", "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99",
            "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]

xname = []
yname = []
color = []
alpha = []
for i, node1 in enumerate(nodes):
    for j, node2 in enumerate(nodes):
        xname.append(node1['name'])
        yname.append(node2['name'])

        alpha.append(min(counts[i,j]/4.0, 0.9) + 0.1)

        if node1['group'] == node2['group']:
            color.append(colormap[node1['group']])
        else:
            color.append('lightgrey')

data=dict(
    xname=xname,
    yname=yname,
    colors=color,
    alphas=alpha,
    count=counts.flatten(),
)

#lasso
s1 = ColumnDataSource(data=dict(x=xname, y=yname))
p1 = figure(title="Les Mis Occurrences",
           x_axis_location="above", tools="hover,save", tools="lasso_select"
           x_range=list(reversed(names)), 
            y_range=names, tooltips = [('names', '@yname, @xname'), ('count', '@count')])
p1.rect('xname', 'yname', 0.9, 0.9, source=s1,
       color='colors', alpha='alphas', line_color=None,
       hover_line_color='black', hover_color='colors')

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=800, plot_height=800, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.rect('x', 'y', source=s2, alpha='alphas')

p1.plot_width = 800
p1.plot_height = 800
p1.grid.grid_line_color = None
p1.axis.axis_line_color = None
p1.axis.major_tick_line_color = None
p1.axis.major_label_text_font_size = "5pt"
p1.axis.major_label_standoff = 0
p1.xaxis.major_label_orientation = np.pi/3

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)


#slider DOES NOT WORK IN ADJACENCY MATRIX. TRY OTHER STUFF LIKE DIFFERENT COLOURS
from bokeh.layouts import column
from bokeh.models import Slider

slider = Slider(start=0.1, end=2, step=0.01, value=0.2)

output_file("les_mis.html", title="les_mis.py example")

layout = row(p1, p2)

show(column(layout, slider)) # show the plot

SyntaxError: invalid syntax (<ipython-input-6-eca355b1040d>, line 59)

In [3]:
#!/usr/bin/python3
from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.models import DatetimeTickFormatter, ColumnDataSource

#Import data-->Weight measurements over a period of time [ STUB ]
weight = pd.read_csv("weight.csv")

#Define parameters
weight["Date"] = pd.to_datetime(weight['Date'])
weight["Weight"] = pd.to_numeric(weight['Weight'])

source = ColumnDataSource(weight)

#Take data  and present in a graph
output_file("test.html")
p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
p.line(x='Date',y='Weight',line_width=2, source=source)
p.xaxis.formatter=DatetimeTickFormatter(
    minutes=["%M"],
    hours=["%H:%M"],
    days=["%d/%m/%Y"],
    months=["%m/%Y"],
    years=["%Y"]
)
show(p)

FileNotFoundError: File b'weight.csv' does not exist